In [1]:
import pandas as pd
import numpy as np

## Load and filter Non-Surgical PDs

In [2]:
file_path = r"C:\Users\Admin\Downloads\ParkinsonsDisease_NeuromodulationResearch\PD_DataForPDbaseline4PropMatch\for_model.csv"
Marius = pd.read_csv(file_path)

# Remove the last 13 columns
Marius_trimmed = Marius.iloc[:, :-13]

# Keep only the 'PATNO', 'ENROLL_AGE', and 'SEX' columns
Marius_relevant = Marius_trimmed[['PATNO', 'ENROLL_AGE', 'SEX']]

# Remove duplicate PATNO values while keeping the first occurrence
Marius_unique = Marius_relevant.drop_duplicates(subset='PATNO', keep='last')

# Print the result
print(Marius_unique.head())
Marius_unique.shape

    PATNO  ENROLL_AGE     SEX
16   3001        65.1    male
29   3002        67.6  female
42   3003        56.7  female
44   3006        57.5  female
60   3010        47.0    male


(1061, 3)

## Load each all UPDRS collected Scores (I-III) and Filter only for Measurments at Baseline (BL) 

In [3]:
# Load and filter dataIII
file_path = r"C:\Users\Admin\Downloads\ParkinsonsDisease_NeuromodulationResearch\PD_DataForPDbaseline4PropMatch\MDS-UPDRS_Part_III_28Aug2024.csv"
data = pd.read_csv(file_path)
dataIII = data[["PATNO", "EVENT_ID", "INFODT", "NP3TOT"]]
dataIII = dataIII[dataIII["EVENT_ID"].str.strip() == "BL"]

# Load and filter dataII
file_path = r"C:\Users\Admin\Downloads\ParkinsonsDisease_NeuromodulationResearch\PD_DataForPDbaseline4PropMatch\MDS_UPDRS_Part_II__Patient_Questionnaire_28Aug2024.csv"
data = pd.read_csv(file_path)
dataII = data[["PATNO", "EVENT_ID", "INFODT", "NP2PTOT"]]
dataII = dataII[dataII["EVENT_ID"].str.strip() == "BL"]

# Load and filter dataIPQ
file_path = r"C:\Users\Admin\Downloads\ParkinsonsDisease_NeuromodulationResearch\PD_DataForPDbaseline4PropMatch\MDS-UPDRS_Part_I_Patient_Questionnaire_28Aug2024.csv"
data = pd.read_csv(file_path)
dataIPQ = data[["PATNO", "EVENT_ID", "INFODT", "NP1PTOT"]]
dataIPQ = dataIPQ[dataIPQ["EVENT_ID"].str.strip() == "BL"]

# Load and filter dataI
file_path = r"C:\Users\Admin\Downloads\ParkinsonsDisease_NeuromodulationResearch\PD_DataForPDbaseline4PropMatch\MDS-UPDRS_Part_I_28Aug2024.csv"
data = pd.read_csv(file_path)
dataI = data[["PATNO", "EVENT_ID", "INFODT", "NP1RTOT"]]
dataI = dataI[dataI["EVENT_ID"].str.strip() == "BL"]

# Display the results
print(dataIII.head())
print(dataII.head())
print(dataIPQ.head())
print(dataI.head())

    PATNO EVENT_ID   INFODT  NP3TOT
0    3000       BL  02/2011     4.0
9    3001       BL  03/2011    12.0
35   3002       BL  03/2011    17.0
61   3003       BL  04/2011    29.0
85   3004       BL  04/2011     2.0
    PATNO EVENT_ID   INFODT  NP2PTOT
0    3000       BL  02/2011      0.0
8    3001       BL  03/2011      2.0
30   3002       BL  03/2011     15.0
50   3003       BL  04/2011      6.0
71   3004       BL  04/2011      0.0
    PATNO EVENT_ID   INFODT  NP1PTOT
0    3000       BL  02/2011      3.0
8    3001       BL  03/2011      8.0
30   3002       BL  03/2011      5.0
50   3003       BL  04/2011     11.0
71   3004       BL  04/2011      2.0
    PATNO EVENT_ID   INFODT  NP1RTOT
0    3000       BL  02/2011      3.0
9    3001       BL  03/2011      0.0
31   3002       BL  03/2011      3.0
51   3003       BL  04/2011      1.0
72   3004       BL  04/2011      0.0


### Merge all filtered UPDRS 

In [4]:
# Merge the datasets on 'PATNO' and 'EVENT_ID' using inner joins
merged_df_31_08 = dataIII.merge(dataII, on=['PATNO', 'EVENT_ID'], how='inner') \
                   .merge(dataIPQ, on=['PATNO', 'EVENT_ID'], how='inner') \
                   .merge(dataI, on=['PATNO', 'EVENT_ID'], how='inner')

print(merged_df_31_08.head())

   PATNO EVENT_ID INFODT_x  NP3TOT INFODT_y  NP2PTOT INFODT_x  NP1PTOT  \
0   3000       BL  02/2011     4.0  02/2011      0.0  02/2011      3.0   
1   3001       BL  03/2011    12.0  03/2011      2.0  03/2011      8.0   
2   3002       BL  03/2011    17.0  03/2011     15.0  03/2011      5.0   
3   3003       BL  04/2011    29.0  04/2011      6.0  04/2011     11.0   
4   3004       BL  04/2011     2.0  04/2011      0.0  04/2011      2.0   

  INFODT_y  NP1RTOT  
0  02/2011      3.0  
1  03/2011      0.0  
2  03/2011      3.0  
3  04/2011      1.0  
4  04/2011      0.0  


In [5]:
# Create the new column 'UPDRSTOT' by summing the specified columns
merged_df_31_08['UPDRSTOT'] = merged_df_31_08[['NP3TOT', 'NP2PTOT', 'NP1PTOT', 'NP1RTOT']].sum(axis=1)
print(merged_df_31_08.head())

   PATNO EVENT_ID INFODT_x  NP3TOT INFODT_y  NP2PTOT INFODT_x  NP1PTOT  \
0   3000       BL  02/2011     4.0  02/2011      0.0  02/2011      3.0   
1   3001       BL  03/2011    12.0  03/2011      2.0  03/2011      8.0   
2   3002       BL  03/2011    17.0  03/2011     15.0  03/2011      5.0   
3   3003       BL  04/2011    29.0  04/2011      6.0  04/2011     11.0   
4   3004       BL  04/2011     2.0  04/2011      0.0  04/2011      2.0   

  INFODT_y  NP1RTOT  UPDRSTOT  
0  02/2011      3.0      10.0  
1  03/2011      0.0      22.0  
2  03/2011      3.0      40.0  
3  04/2011      1.0      47.0  
4  04/2011      0.0       4.0  


In [6]:
merged_df_31_08.shape

(3922, 11)

In [7]:
merged_df_31_08['PATNO'].nunique()

3744

In [8]:
# Define the required columns, keeping only the first occurrence of INFODT_x and renaming it to INFODT
required_columns = ['PATNO', 'EVENT_ID', 'INFODT_x', 'NP3TOT', 'NP2PTOT', 'NP1PTOT', 'NP1RTOT', 'UPDRSTOT']

# Select the required columns from the DataFrame
merged_df_31_08_dropped = merged_df_31_08[required_columns]

# Rename 'INFODT_x' to 'INFODT'
merged_df_31_08_dropped = merged_df_31_08_dropped.rename(columns={'INFODT_x': 'INFODT'})

# Drop any remaining occurrences of 'INFODT' if they exist
merged_df_31_08_dropped = merged_df_31_08_dropped.loc[:, ~merged_df_31_08_dropped.columns.duplicated()]

# Display the result
print(merged_df_31_08_dropped.head())
merged_df_31_08_dropped.shape

   PATNO EVENT_ID   INFODT  NP3TOT  NP2PTOT  NP1PTOT  NP1RTOT  UPDRSTOT
0   3000       BL  02/2011     4.0      0.0      3.0      3.0      10.0
1   3001       BL  03/2011    12.0      2.0      8.0      0.0      22.0
2   3002       BL  03/2011    17.0     15.0      5.0      3.0      40.0
3   3003       BL  04/2011    29.0      6.0     11.0      1.0      47.0
4   3004       BL  04/2011     2.0      0.0      2.0      0.0       4.0


(3922, 8)

In [9]:
merged_df_31_08_unique = merged_df_31_08_dropped.drop_duplicates(subset='PATNO')

In [10]:
merged_df_31_08_unique.shape

(3744, 8)

In [11]:
merged_df_31_08_unique.head()

,PATNO,EVENT_ID,INFODT,NP3TOT,NP2PTOT,NP1PTOT,NP1RTOT,UPDRSTOT
0,3000,BL,02/2011,4.0,0.0,3.0,3.0,10.0
1,3001,BL,03/2011,12.0,2.0,8.0,0.0,22.0
2,3002,BL,03/2011,17.0,15.0,5.0,3.0,40.0
3,3003,BL,04/2011,29.0,6.0,11.0,1.0,47.0
4,3004,BL,04/2011,2.0,0.0,2.0,0.0,4.0


### Merge based on Non-surgical patients

In [12]:
# Perform the inner join on 'PATNO'
final_merged_df = merged_df_31_08_unique.merge(Marius_unique, on='PATNO', how='inner')

# Display the result
print(final_merged_df.head())

   PATNO EVENT_ID   INFODT  NP3TOT  NP2PTOT  NP1PTOT  NP1RTOT  UPDRSTOT  \
0   3001       BL  03/2011    12.0      2.0      8.0      0.0      22.0   
1   3002       BL  03/2011    17.0     15.0      5.0      3.0      40.0   
2   3003       BL  04/2011    29.0      6.0     11.0      1.0      47.0   
3   3006       BL  04/2011    22.0      4.0      2.0      1.0      29.0   
4   3010       BL  06/2011    19.0     11.0      9.0      2.0      41.0   

   ENROLL_AGE     SEX  
0        65.1    male  
1        67.6  female  
2        56.7  female  
3        57.5  female  
4        47.0    male  


In [13]:
# Check if all PATNO values in final_merged_df are present in Marius_unique
all_match = final_merged_df['PATNO'].isin(Marius_unique['PATNO']).all()

if all_match:
    print("All PATNO values in final_merged_df match those in Marius_unique.")
else:
    print("There are PATNO values in final_merged_df that do not match those in Marius_unique.")

All PATNO values in final_merged_df match those in Marius_unique.


In [14]:
assert final_merged_df['PATNO'].isin(Marius_unique['PATNO']).all(), "There are PATNO values in final_merged_df that do not match those in Marius_unique."

In [15]:
print(final_merged_df.shape)
final_merged_df.isna().sum()
final_merged_df.isna().sum()

(1058, 10)


PATNO          0
EVENT_ID       0
INFODT         0
NP3TOT         4
NP2PTOT        1
NP1PTOT        0
NP1RTOT        4
UPDRSTOT       0
ENROLL_AGE    11
SEX            0
dtype: int64

In [16]:
final_merged_df['PATNO'].nunique()

1058

In [17]:
# Remove all rows with any NaN values
final_merged_df_cleaned = final_merged_df.dropna()
print(final_merged_df_cleaned.isna().sum())
final_merged_df_cleaned.shape

PATNO         0
EVENT_ID      0
INFODT        0
NP3TOT        0
NP2PTOT       0
NP1PTOT       0
NP1RTOT       0
UPDRSTOT      0
ENROLL_AGE    0
SEX           0
dtype: int64


(1038, 10)

In [18]:
final_merged_df_cleaned['PATNO'].nunique()

1038

### EXPLORE UPDRS IV data

In [19]:
# Load and filter dataIV
file_path = r"C:\Users\Admin\Downloads\ParkinsonsDisease_NeuromodulationResearch\PD_DataForPDbaseline4PropMatch\MDS-UPDRS_Part_IV__Motor_Complications_28Aug2024.csv"
data = pd.read_csv(file_path)
dataIV = data[["PATNO", "EVENT_ID", "NP4TOT"]]
dataIV= dataIV[dataIV["EVENT_ID"].str.strip() == "BL"]

In [20]:
# Find the PATNO values that are common to both dataIV and final_merged_df_cleaned
common_patno = dataIV['PATNO'].isin(final_merged_df_cleaned['PATNO'])

# Count the number of matching PATNO values
num_matching_patno = common_patno.sum()

print(f"The number of PATNO values in dataIV that match those in final_merged_df_cleaned is: {num_matching_patno}")

The number of PATNO values in dataIV that match those in final_merged_df_cleaned is: 241


In [21]:
# Find the PATNO values that are common to both dataIV and final_merged_df_cleaned
matching_patno = dataIV[dataIV['PATNO'].isin(final_merged_df_cleaned['PATNO'])]

# Count how many of these have NP4TOT == 0
count_np4tot_zero = (matching_patno['NP4TOT'] == 0).sum()

print(f"The number of matching PATNO values in dataIV with NP4TOT == 0 is: {count_np4tot_zero}")

The number of matching PATNO values in dataIV with NP4TOT == 0 is: 150


##### _Merging the UPDRS-IV will compromise the collected non-surgical data due to very large missing values, therefore was execluded from total UPDRS calculation_

### Demographics + PD Diagnosis Date Merging

In [22]:
# Load and prepare Demo DataFrame
file_path = r"C:\Users\Admin\Downloads\ParkinsonsDisease_NeuromodulationResearch\PD_DataForPDbaseline4PropMatch\Demographics_31Aug2024.csv"
Demo = pd.read_csv(file_path)

# Keep only the 'PATNO', 'BIRTHDT', 'HANDED' columns
Demo_relevant = Demo[['PATNO', 'BIRTHDT', 'HANDED']]

# Filter Demo_relevant to keep only PATNO values present in final_merged_df_cleaned['PATNO']
filtered_Demo_relevant = Demo_relevant[Demo_relevant['PATNO'].isin(final_merged_df_cleaned['PATNO'])]

# Display the result
print(filtered_Demo_relevant.head(6))

    PATNO  BIRTHDT  HANDED
1    3001  01/1946     2.0
2    3002  08/1943     1.0
3    3003  07/1954     1.0
6    3006  10/1953     1.0
10   3010  06/1964     3.0
12   3012  01/1953     1.0


In [23]:
filtered_Demo_relevant.shape

(1038, 3)

In [24]:
filtered_Demo_relevant.isna().sum()

PATNO      0
BIRTHDT    0
HANDED     1
dtype: int64

In [25]:
# Load and prepare Demo DataFrame
file_path = r"C:\Users\Admin\Downloads\ParkinsonsDisease_NeuromodulationResearch\PD_DataForPDbaseline4PropMatch\PD_Diagnosis_History_31Aug2024.csv"
PDDX = pd.read_csv(file_path)

# Keep only the 'PATNO', 'PDDXDT' columns
PDDX_relevant = PDDX[['PATNO', 'PDDXDT']]

# Filter Demo_relevant to keep only PATNO values present in final_merged_df_cleaned['PATNO']
filtered_PDDX_relevant = PDDX_relevant[PDDX_relevant['PATNO'].isin(final_merged_df_cleaned['PATNO'])]

# Display the result
print(filtered_PDDX_relevant.head(6))

   PATNO   PDDXDT
0   3001  04/2010
1   3002  02/2010
2   3003  03/2009
4   3006  11/2010
6   3010  02/2011
7   3012  08/2011


In [26]:
filtered_PDDX_relevant.shape

(1037, 2)

In [27]:
# Merge both datasets on 'PATNO' to retain only shared PATNO values
merged_filtered_DemoPDDX = pd.merge(filtered_Demo_relevant, filtered_PDDX_relevant, on='PATNO', how='inner')

# Display the merged result
print("Merged DataFrame:")
print(merged_filtered_DemoPDDX.head(6))
merged_filtered_DemoPDDX.shape

Merged DataFrame:
   PATNO  BIRTHDT  HANDED   PDDXDT
0   3001  01/1946     2.0  04/2010
1   3002  08/1943     1.0  02/2010
2   3003  07/1954     1.0  03/2009
3   3006  10/1953     1.0  11/2010
4   3010  06/1964     3.0  02/2011
5   3012  01/1953     1.0  08/2011


(1037, 4)

In [28]:
merged_filtered_DemoPDDX.isna().sum()

PATNO      0
BIRTHDT    0
HANDED     1
PDDXDT     0
dtype: int64

In [29]:
# Convert 'BIRTHDT' and 'PDDXDT' to datetime objects
merged_filtered_DemoPDDX['BIRTHDT'] = pd.to_datetime(merged_filtered_DemoPDDX['BIRTHDT'], format='%m/%Y')
merged_filtered_DemoPDDX['PDDXDT'] = pd.to_datetime(merged_filtered_DemoPDDX['PDDXDT'], format='%m/%Y')

# Calculate the difference in years
merged_filtered_DemoPDDX['AgePDDX'] = (merged_filtered_DemoPDDX['PDDXDT'] - merged_filtered_DemoPDDX['BIRTHDT']).astype('<m8[Y]')

# Display the DataFrame with the new 'AgePDDX' column
print(merged_filtered_DemoPDDX)

       PATNO    BIRTHDT  HANDED     PDDXDT  AgePDDX
0       3001 1946-01-01     2.0 2010-04-01     64.0
1       3002 1943-08-01     1.0 2010-02-01     66.0
2       3003 1954-07-01     1.0 2009-03-01     54.0
3       3006 1953-10-01     1.0 2010-11-01     57.0
4       3010 1964-06-01     3.0 2011-02-01     46.0
...      ...        ...     ...        ...      ...
1032  293681 1943-12-01     1.0 2023-11-01     79.0
1033  293787 1978-07-01     2.0 2023-08-01     45.0
1034  293795 1948-01-01     1.0 2023-11-01     75.0
1035  293798 1952-06-01     1.0 2023-11-01     71.0
1036  293800 1959-08-01     1.0 2023-05-01     63.0

[1037 rows x 5 columns]


### MOCA Filtering and Merging with the merged Demo/PDDX

#### _IF Baseline (BL) measure is not present, the measure at Screening (SC) was taken instead_

In [30]:
# Load the MOCA DataFrame
file_path = r"C:\Users\Admin\Downloads\ParkinsonsDisease_NeuromodulationResearch\PD_DataForPDbaseline4PropMatch\Montreal_Cognitive_Assessment__MoCA__29Aug2024.csv"
MOCA = pd.read_csv(file_path)

# Keep only the 'PATNO', 'EVENT_ID', 'MCATOT' columns
MOCA_relevant = MOCA[['PATNO', 'EVENT_ID', 'MCATOT']]

# Filter to keep only PATNO values with EVENT_ID = BL
BL_rows = MOCA_relevant[MOCA_relevant['EVENT_ID'].str.strip() == 'BL']

# Get PATNO values that have EVENT_ID = BL
BL_patnos = BL_rows['PATNO'].unique()

# Filter to keep only PATNO with EVENT_ID = SC for those not already in BL
SC_rows = MOCA_relevant[(MOCA_relevant['EVENT_ID'].str.strip() == 'SC') & (~MOCA_relevant['PATNO'].isin(BL_patnos))]

# Further filter both BL_rows and SC_rows to include only PATNO present in final_merged_df_cleaned['PATNO']
final_BL_rows = BL_rows[BL_rows['PATNO'].isin(final_merged_df_cleaned['PATNO'])]
final_SC_rows = SC_rows[SC_rows['PATNO'].isin(final_merged_df_cleaned['PATNO'])]

# Combine both results
filtered_MOCA_relevant = pd.concat([final_BL_rows, final_SC_rows])

# Display the result
print(filtered_MOCA_relevant.head(20))

        PATNO EVENT_ID  MCATOT
6112    40784       BL    24.0
6114    40789       BL    24.0
6140    40893       BL    29.0
6407    41411       BL    16.0
11100  143757       BL    29.0
11366  156490       BL    27.0
11553  163478       BL    30.0
11578  164985       BL    26.0
11707  170471       BL    30.0
11712  170481       BL    29.0
11781  173006       BL    28.0
11786  173327       BL    27.0
12079  182984       BL    30.0
12102  184190       BL    28.0
12138  186108       BL    30.0
12174  188170       BL    30.0
12227  194803       BL    27.0
12271  198746       BL    23.0
12374  208996       BL    30.0
12377  209141       BL    29.0


In [31]:
filtered_MOCA_relevant.shape

(1037, 3)

In [32]:
# Check for duplicate PATNO values
duplicate_patnos = filtered_MOCA_relevant[filtered_MOCA_relevant.duplicated(subset='PATNO', keep=False)]
print("Duplicates based on PATNO:")
print(duplicate_patnos)

# Count of duplicate PATNO values
duplicate_count = filtered_MOCA_relevant['PATNO'].duplicated().sum()
print(f"Number of duplicate PATNO values: {duplicate_count}")

Duplicates based on PATNO:
Empty DataFrame
Columns: [PATNO, EVENT_ID, MCATOT]
Index: []
Number of duplicate PATNO values: 0


In [33]:
# Check for NaN values in each column
nan_counts = filtered_MOCA_relevant.isna().sum()
print("Count of NaN values in each column:")
print(nan_counts)

# Check if there are any NaN values in the DataFrame
has_nan = filtered_MOCA_relevant.isna().any().any()
print(f"Are there any NaN values in the DataFrame? {'Yes' if has_nan else 'No'}")

Count of NaN values in each column:
PATNO       0
EVENT_ID    0
MCATOT      1
dtype: int64
Are there any NaN values in the DataFrame? Yes


#### _Merge MOCA with Demo/PDDX_

In [34]:
# Perform an inner join on 'PATNO' to keep only the shared PATNO values
MOCA_DemoPDDX = pd.merge(merged_filtered_DemoPDDX, filtered_MOCA_relevant, on='PATNO', how='inner')

print(MOCA_DemoPDDX.head())
print(f"Number of rows in merged DataFrame: {MOCA_DemoPDDX.shape[0]}")

   PATNO    BIRTHDT  HANDED     PDDXDT  AgePDDX EVENT_ID  MCATOT
0   3001 1946-01-01     2.0 2010-04-01     64.0       SC    29.0
1   3002 1943-08-01     1.0 2010-02-01     66.0       SC    29.0
2   3003 1954-07-01     1.0 2009-03-01     54.0       SC    25.0
3   3006 1953-10-01     1.0 2010-11-01     57.0       SC    27.0
4   3010 1964-06-01     3.0 2011-02-01     46.0       SC    26.0
Number of rows in merged DataFrame: 1036


In [35]:
MOCA_DemoPDDX_cleaned = MOCA_DemoPDDX.dropna()

In [36]:
MOCA_DemoPDDX_cleaned.isna().sum().sum()

0

In [37]:
MOCA_DemoPDDX_cleaned.shape

(1034, 7)

### Merge MOCA_DemoPDDX and final_merged_df_cleaned (UPDRS total)

In [38]:
# Perform an inner join on 'PATNO' to keep only the shared PATNO values
MOCA_DemoPDDX_UPDRS = pd.merge(MOCA_DemoPDDX, final_merged_df_cleaned, on='PATNO', how='inner')

# Display the result
print(MOCA_DemoPDDX_UPDRS.head())

# Optional: Check the number of rows in the merged DataFrame
print(f"Number of rows in merged DataFrame: {MOCA_DemoPDDX_UPDRS.shape[0]}")

   PATNO    BIRTHDT  HANDED     PDDXDT  AgePDDX EVENT_ID_x  MCATOT EVENT_ID_y  \
0   3001 1946-01-01     2.0 2010-04-01     64.0         SC    29.0         BL   
1   3002 1943-08-01     1.0 2010-02-01     66.0         SC    29.0         BL   
2   3003 1954-07-01     1.0 2009-03-01     54.0         SC    25.0         BL   
3   3006 1953-10-01     1.0 2010-11-01     57.0         SC    27.0         BL   
4   3010 1964-06-01     3.0 2011-02-01     46.0         SC    26.0         BL   

    INFODT  NP3TOT  NP2PTOT  NP1PTOT  NP1RTOT  UPDRSTOT  ENROLL_AGE     SEX  
0  03/2011    12.0      2.0      8.0      0.0      22.0        65.1    male  
1  03/2011    17.0     15.0      5.0      3.0      40.0        67.6  female  
2  04/2011    29.0      6.0     11.0      1.0      47.0        56.7  female  
3  04/2011    22.0      4.0      2.0      1.0      29.0        57.5  female  
4  06/2011    19.0     11.0      9.0      2.0      41.0        47.0    male  
Number of rows in merged DataFrame: 1036


In [39]:
MOCA_DemoPDDX_UPDRS.isna().sum()

PATNO         0
BIRTHDT       0
HANDED        1
PDDXDT        0
AgePDDX       0
EVENT_ID_x    0
MCATOT        1
EVENT_ID_y    0
INFODT        0
NP3TOT        0
NP2PTOT       0
NP1PTOT       0
NP1RTOT       0
UPDRSTOT      0
ENROLL_AGE    0
SEX           0
dtype: int64

In [40]:
MOCA_DemoPDDX_UPDRS['PATNO'].nunique()

1036

In [41]:
# Check for duplicate PATNO values
duplicates = MOCA_DemoPDDX_UPDRS['PATNO'].duplicated().sum()
print(f"Number of duplicate PATNO values: {duplicates}")

# Check for NaN values in each column
nan_summary = MOCA_DemoPDDX_UPDRS.isna().sum()
print("NaN values in each column:")
print(nan_summary)

# Display rows with NaN values (if any)
nan_rows = MOCA_DemoPDDX_UPDRS[MOCA_DemoPDDX_UPDRS.isna().any(axis=1)]
print("Rows with NaN values:")
print(nan_rows)

Number of duplicate PATNO values: 0
NaN values in each column:
PATNO         0
BIRTHDT       0
HANDED        1
PDDXDT        0
AgePDDX       0
EVENT_ID_x    0
MCATOT        1
EVENT_ID_y    0
INFODT        0
NP3TOT        0
NP2PTOT       0
NP1PTOT       0
NP1RTOT       0
UPDRSTOT      0
ENROLL_AGE    0
SEX           0
dtype: int64
Rows with NaN values:
      PATNO    BIRTHDT  HANDED     PDDXDT  AgePDDX EVENT_ID_x  MCATOT  \
570   50157 1953-07-01     1.0 2012-03-01     58.0         SC     NaN   
818  140041 1966-08-01     NaN 2019-12-01     53.0         SC    28.0   

    EVENT_ID_y   INFODT  NP3TOT  NP2PTOT  NP1PTOT  NP1RTOT  UPDRSTOT  \
570         BL  10/2014    15.0      7.0     13.0      2.0      37.0   
818         BL  03/2022    22.0      8.0     11.0      4.0      45.0   

     ENROLL_AGE     SEX  
570        61.2  female  
818        56.1  female  


In [42]:
MOCA_DemoPDDX_UPDRS = MOCA_DemoPDDX_UPDRS.dropna()
MOCA_DemoPDDX_UPDRS.shape

(1034, 16)

In [43]:
MOCA_DemoPDDX_UPDRS.head(5)

,PATNO,BIRTHDT,HANDED,PDDXDT,AgePDDX,EVENT_ID_x,MCATOT,EVENT_ID_y,INFODT,NP3TOT,NP2PTOT,NP1PTOT,NP1RTOT,UPDRSTOT,ENROLL_AGE,SEX
0,3001,1946-01-01,2.0,2010-04-01,64.0,SC,29.0,BL,03/2011,12.0,2.0,8.0,0.0,22.0,65.1,male
1,3002,1943-08-01,1.0,2010-02-01,66.0,SC,29.0,BL,03/2011,17.0,15.0,5.0,3.0,40.0,67.6,female
2,3003,1954-07-01,1.0,2009-03-01,54.0,SC,25.0,BL,04/2011,29.0,6.0,11.0,1.0,47.0,56.7,female
3,3006,1953-10-01,1.0,2010-11-01,57.0,SC,27.0,BL,04/2011,22.0,4.0,2.0,1.0,29.0,57.5,female
4,3010,1964-06-01,3.0,2011-02-01,46.0,SC,26.0,BL,06/2011,19.0,11.0,9.0,2.0,41.0,47.0,male


In [44]:
# Rename 'EVENT_ID_y' to 'EVENT_ID'
MOCA_DemoPDDX_UPDRS = MOCA_DemoPDDX_UPDRS.rename(columns={'EVENT_ID_y': 'EVENT_ID'})

# Define the desired column order, now including 'EVENT_ID'
desired_order = [
    'PATNO', 'EVENT_ID', 'INFODT', 'BIRTHDT', 'SEX', 'HANDED', 'PDDXDT', 
    'AgePDDX', 'ENROLL_AGE', 'MCATOT', 'NP3TOT', 'NP2PTOT', 
    'NP1PTOT', 'NP1RTOT', 'UPDRSTOT'
]

# Reorder columns
MOCA_DemoPDDX_UPDRS = MOCA_DemoPDDX_UPDRS[desired_order]

In [45]:
# Ensure that 'PDDXDT' and 'INFODT' are in datetime format
MOCA_DemoPDDX_UPDRS['PDDXDT'] = pd.to_datetime(MOCA_DemoPDDX_UPDRS['PDDXDT'], format='%m/%Y')
MOCA_DemoPDDX_UPDRS['INFODT'] = pd.to_datetime(MOCA_DemoPDDX_UPDRS['INFODT'], format='%m/%Y')

# Subtract 'PDDXDT' from 'INFODT' to calculate the time difference in days
MOCA_DemoPDDX_UPDRS['PDTIME(Days)'] = (MOCA_DemoPDDX_UPDRS['INFODT'] - MOCA_DemoPDDX_UPDRS['PDDXDT']).dt.days

MOCA_DemoPDDX_UPDRS.head()

,PATNO,EVENT_ID,INFODT,BIRTHDT,SEX,HANDED,PDDXDT,AgePDDX,ENROLL_AGE,MCATOT,NP3TOT,NP2PTOT,NP1PTOT,NP1RTOT,UPDRSTOT,PDTIME(Days)
0,3001,BL,2011-03-01,1946-01-01,male,2.0,2010-04-01,64.0,65.1,29.0,12.0,2.0,8.0,0.0,22.0,334
1,3002,BL,2011-03-01,1943-08-01,female,1.0,2010-02-01,66.0,67.6,29.0,17.0,15.0,5.0,3.0,40.0,393
2,3003,BL,2011-04-01,1954-07-01,female,1.0,2009-03-01,54.0,56.7,25.0,29.0,6.0,11.0,1.0,47.0,761
3,3006,BL,2011-04-01,1953-10-01,female,1.0,2010-11-01,57.0,57.5,27.0,22.0,4.0,2.0,1.0,29.0,151
4,3010,BL,2011-06-01,1964-06-01,male,3.0,2011-02-01,46.0,47.0,26.0,19.0,11.0,9.0,2.0,41.0,120


In [46]:
# Define the desired column order, now including 'EVENT_ID'
desired_order = [
    'PATNO', 'EVENT_ID', 'INFODT', 'BIRTHDT', 'SEX', 'HANDED', 'PDDXDT', 
    'AgePDDX', 'ENROLL_AGE', 'PDTIME(Days)', 'MCATOT', 'NP3TOT', 'NP2PTOT', 
    'NP1PTOT', 'NP1RTOT', 'UPDRSTOT'
]

# Reorder columns
MOCA_DemoPDDX_UPDRS = MOCA_DemoPDDX_UPDRS[desired_order]

In [47]:
MOCA_DemoPDDX_UPDRS.head()

,PATNO,EVENT_ID,INFODT,BIRTHDT,SEX,HANDED,PDDXDT,AgePDDX,ENROLL_AGE,PDTIME(Days),MCATOT,NP3TOT,NP2PTOT,NP1PTOT,NP1RTOT,UPDRSTOT
0,3001,BL,2011-03-01,1946-01-01,male,2.0,2010-04-01,64.0,65.1,334,29.0,12.0,2.0,8.0,0.0,22.0
1,3002,BL,2011-03-01,1943-08-01,female,1.0,2010-02-01,66.0,67.6,393,29.0,17.0,15.0,5.0,3.0,40.0
2,3003,BL,2011-04-01,1954-07-01,female,1.0,2009-03-01,54.0,56.7,761,25.0,29.0,6.0,11.0,1.0,47.0
3,3006,BL,2011-04-01,1953-10-01,female,1.0,2010-11-01,57.0,57.5,151,27.0,22.0,4.0,2.0,1.0,29.0
4,3010,BL,2011-06-01,1964-06-01,male,3.0,2011-02-01,46.0,47.0,120,26.0,19.0,11.0,9.0,2.0,41.0


In [48]:
file_path = r"C:\Users\Admin\Downloads\ParkinsonsDisease_NeuromodulationResearch\PPMI_PDBL_MOCA_DemoPDDX_UPDRS_4PropensityM.csv"

# Save the DataFrame to a CSV file
MOCA_DemoPDDX_UPDRS.to_csv(file_path, index=False)


print("This is the final PD at baseline dataset for further propensity matching")
MOCA_DemoPDDX_UPDRS.head(10)
print(f"DataFrame saved to {file_path}")

This is the final PD at baseline dataset for further propensity matching
DataFrame saved to C:\Users\Admin\Downloads\ParkinsonsDisease_NeuromodulationResearch\PPMI_PDBL_MOCA_DemoPDDX_UPDRS_4PropensityM.csv


## Propensity Matching

### File 1 Preparation

In [49]:
file_path = r"C:\Users\Admin\Downloads\ParkinsonsDisease_NeuromodulationResearch\PD_DataForPDbaseline4PropMatch\OurPDDataforPM.xlsx"
excel_data = pd.read_excel(file_path)

In [50]:
# Filter rows where EVENT_TAG is "BL" and MEDICATION is "OFF"
filtered_data = excel_data[(excel_data['EVENT_TAG'] == 'BL') & (excel_data['MEDICATION'] == 'OFF')]
filtered_data.head()

,patid,Improver(1) - Decliner (0),MoCA_PRE,MDS-UPDRS_3_ONLY_Gesamt,EVENT_TAG,MEDICATION,STIMULATION,MoCA_POST,SEX,DISEASE_DURATION_AT_OP,AGE_AT_DIAGNOSIS,AGE_AT_OP,surgeryday
336,2018-PD1,NaN,NaN,29.0,BL,OFF,OFF,NaN,M,8.087611,41.864476,49.952088,2018-02-02
341,2018-PD10,1.0,28.0,28.0,BL,OFF,OFF,30.0,M,5.144422,45.418207,50.562628,2018-02-23
348,2018-PD11,NaN,28.0,70.0,BL,OFF,OFF,28.0,M,7.192334,61.664613,68.856947,2018-03-12
356,2018-PD3,1.0,29.0,57.0,BL,OFF,OFF,30.0,M,7.260780,54.151951,61.412731,2018-04-06
364,2018-PD4,NaN,NaN,35.0,BL,OFF,OFF,NaN,M,10.280630,40.558522,50.839151,2018-04-13


In [51]:
# Define the column rename mapping
column_rename_mapping = {
    'MoCA_PRE': 'MCATOT',
    'MDS-UPDRS_3_ONLY_Gesamt': 'NP3TOT',
    'DISEASE_DURATION_AT_OP': 'PDTIME(Years)',
    'AGE_AT_DIAGNOSIS': 'AgePDDX',
    'SEX': 'SEX'   
}

# Rename the columns using the rename method
OurPDDataforPM = filtered_data.rename(columns=column_rename_mapping)

# Exclude remaining columns that are not in the column_rename_mapping
# Get the list of new column names from the rename mapping
new_columns = list(column_rename_mapping.values())

# Keep only the columns that are in the new column names list
OurPDDataforPM_W_MatV_Only = OurPDDataforPM[new_columns]

OurPDDataforPM_W_MatV_Only.head()

,MCATOT,NP3TOT,PDTIME(Years),AgePDDX,SEX
336,NaN,29.0,8.087611,41.864476,M
341,28.0,28.0,5.144422,45.418207,M
348,28.0,70.0,7.192334,61.664613,M
356,29.0,57.0,7.260780,54.151951,M
364,NaN,35.0,10.280630,40.558522,M


In [52]:
OurPDDataforPM.isna().sum()

patid                          7
Improver(1) - Decliner (0)    47
MCATOT                        36
NP3TOT                         0
EVENT_TAG                      0
MEDICATION                     0
STIMULATION                    0
MoCA_POST                     36
SEX                            0
PDTIME(Years)                  0
AgePDDX                        0
AGE_AT_OP                      0
surgeryday                     0
dtype: int64

In [53]:
OurPDDataforPM_W_MatV_Only_nandropped = OurPDDataforPM_W_MatV_Only.dropna()

In [54]:
OurPDDataforPM_W_MatV_Only_nandropped.isna().sum()

MCATOT           0
NP3TOT           0
PDTIME(Years)    0
AgePDDX          0
SEX              0
dtype: int64

In [55]:
OurPDDataforPM_W_MatV_Only_nandropped.shape

(73, 5)

### File 2 Preparation

In [56]:
# Define the mapping for replacing values in the 'SEX' column
sex_mapping = {
    'male': 'M',
    'female': 'F'
}

# Replace values in the 'SEX' column using the mapping
MOCA_DemoPDDX_UPDRS['SEX'] = MOCA_DemoPDDX_UPDRS['SEX'].replace(sex_mapping)

# Rename the column
MOCA_DemoPDDX_UPDRS = MOCA_DemoPDDX_UPDRS.rename(columns={'PDTIME(Days)': 'PDTIME(Years)'})

# Convert the values from days to years
MOCA_DemoPDDX_UPDRS['PDTIME(Years)'] = MOCA_DemoPDDX_UPDRS['PDTIME(Years)'] / 365

# Keep only the columns that are in the new column names list
MOCA_DemoPDDX_UPDRS_W_MatV_Only = MOCA_DemoPDDX_UPDRS[new_columns]

MOCA_DemoPDDX_UPDRS_W_MatV_Only.head()

,MCATOT,NP3TOT,PDTIME(Years),AgePDDX,SEX
0,29.0,12.0,0.915068,64.0,M
1,29.0,17.0,1.076712,66.0,F
2,25.0,29.0,2.084932,54.0,F
3,27.0,22.0,0.413699,57.0,F
4,26.0,19.0,0.328767,46.0,M


In [57]:
# Ensure both datasets have the matching variables
matching_vars = ['NP3TOT', 'PDTIME(Years)', 'AgePDDX', 'SEX', 'MCATOT']

# Ensure necessary columns are present
assert all(var in OurPDDataforPM_W_MatV_Only_nandropped.columns for var in matching_vars), "Missing columns in OurPDDataforPM"
assert all(var in MOCA_DemoPDDX_UPDRS_W_MatV_Only.columns for var in matching_vars), "Missing columns in MOCA_DemoPDDX_UPDRS"

### Start performing Propensity Matching

#### Using Sklearn library (Stephan's code) + minor modifications from myside

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, LabelEncoder

continuous_vars = [var for var in matching_vars if var != 'SEX']
# Ensure continuous variables are numeric
for var in continuous_vars:
    OurPDDataforPM_W_MatV_Only_nandropped[var] = pd.to_numeric(OurPDDataforPM_W_MatV_Only_nandropped[var], errors='coerce')
    MOCA_DemoPDDX_UPDRS_W_MatV_Only[var] = pd.to_numeric(MOCA_DemoPDDX_UPDRS_W_MatV_Only[var], errors='coerce')

# Combine the datasets and create a label column
OurPDDataforPM_W_MatV_Only_nandropped['Label'] = 1  # Treatment  
MOCA_DemoPDDX_UPDRS_W_MatV_Only['Label'] = 0  # Control

# Encode categorical variables if needed
le = LabelEncoder()
OurPDDataforPM_W_MatV_Only_nandropped['SEX'] = le.fit_transform(OurPDDataforPM_W_MatV_Only_nandropped['SEX'])
MOCA_DemoPDDX_UPDRS_W_MatV_Only['SEX'] = le.transform(MOCA_DemoPDDX_UPDRS_W_MatV_Only['SEX'])

df_combined = pd.concat([OurPDDataforPM_W_MatV_Only_nandropped, MOCA_DemoPDDX_UPDRS_W_MatV_Only])

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [ ]:
# Save MOCA_DemoPDDX_UPDRS_W_MatV_Only as a CSV file
MOCA_DemoPDDX_UPDRS_W_MatV_Only.to_csv('MOCA_DemoPDDX_UPDRS_W_MatV_Only.csv', index=False)

# Save OurPDDataforPM_W_MatV_Only_nandropped as a CSV file
OurPDDataforPM_W_MatV_Only_nandropped.to_csv('OurPDDataforPM_W_MatV_Only_nandropped.csv', index=False)

In [59]:
df_combined.head()

,MCATOT,NP3TOT,PDTIME(Years),AgePDDX,SEX,Label
341,28.0,28.0,5.144422,45.418207,1,1
348,28.0,70.0,7.192334,61.664613,1,1
356,29.0,57.0,7.260780,54.151951,1,1
370,27.0,31.0,10.299795,62.967830,1,1
376,28.0,30.0,11.375770,47.222450,1,1


In [60]:
# Standardize the continuous variables (age, UPDRS III, etc.)
scaler = StandardScaler()
df_combined[matching_vars[:-2]] = scaler.fit_transform(df_combined[matching_vars[:-2]])

# Logistic regression model to compute propensity scores
model = LogisticRegression()
df_combined['propensity_score'] = model.fit(df_combined[matching_vars], df_combined['Label']).predict_proba(df_combined[matching_vars])[:, 1]

# Nearest Neighbor Matching
nn = NearestNeighbors(n_neighbors=1)
df1_matched_indices = nn.fit(df_combined[df_combined['Label'] == 1]['propensity_score'].values.reshape(-1, 1)).kneighbors(df_combined[df_combined['Label'] == 0]['propensity_score'].values.reshape(-1, 1), return_distance=False).flatten()

# Get matched data
matched_df2 = df_combined[df_combined['Label'] == 1].iloc[df1_matched_indices]
matched_df1 = df_combined[df_combined['Label'] == 0]

# Combine the matched data
matched_data = pd.concat([matched_df1, matched_df2])

In [61]:
matched_data.head(20)

,MCATOT,NP3TOT,PDTIME(Years),AgePDDX,SEX,Label,propensity_score
0,29.0,-0.946150,-0.338263,0.344865,1,0,0.003128
1,29.0,-0.517525,-0.281667,0.544148,0,0,0.002460
2,25.0,0.511175,0.071336,-0.651549,0,0,0.004215
3,27.0,-0.088900,-0.513806,-0.352624,0,0,0.001138
4,26.0,-0.346075,-0.543543,-1.448680,1,0,0.001115
5,27.0,-0.260350,-0.570402,-0.252983,1,0,0.002010
6,25.0,1.111249,-0.571361,0.643790,1,0,0.006290
7,23.0,-0.346075,-0.601098,-0.053700,0,0,0.000378
8,28.0,-0.260350,-0.571361,1.241638,0,0,0.001723
9,29.0,-0.946150,-0.601098,0.245224,0,0,0.000768


In [62]:
matched_data.shape

(2068, 7)

### Evaluate Matched Data

In [63]:
# Mean of continuous variables before matching
means_before = df_combined[df_combined['Label'] == 1][continuous_vars].mean() - df_combined[df_combined['Label'] == 0][continuous_vars].mean()

print("Mean differences before matching:")
print(means_before)

# Matched data
matched_data = pd.concat([matched_df1, matched_df2])

# Mean of continuous variables after matching
means_after = matched_data[matched_data['Label'] == 1][continuous_vars].mean() - matched_data[matched_data['Label'] == 0][continuous_vars].mean()

print("Mean differences after matching:")
print(means_after)


Mean differences before matching:
NP3TOT           1.700135
PDTIME(Years)    2.974388
AgePDDX         -0.579764
MCATOT           0.507061
dtype: float64
Mean differences after matching:
NP3TOT          -0.752042
PDTIME(Years)    0.680503
AgePDDX         -0.853976
MCATOT          -3.157640
dtype: float64


In [64]:
def standardized_mean_difference(df, treatment_label, control_label, covariates):
    treatment = df[df['Label'] == treatment_label][covariates]
    control = df[df['Label'] == control_label][covariates]
    
    mean_treatment = treatment.mean()
    mean_control = control.mean()
    
    std_treatment = treatment.std()
    std_control = control.std()
    
    standardized_diff = (mean_treatment - mean_control) / ((std_treatment + std_control) / 2)
    
    return standardized_diff

# Calculate standardized mean differences for matched data
std_mean_diff = standardized_mean_difference(matched_data, treatment_label=1, control_label=0, covariates=continuous_vars)
print("Standardized Mean Differences after Matching:")
print(std_mean_diff)

Standardized Mean Differences after Matching:
NP3TOT          -0.882683
PDTIME(Years)    1.233220
AgePDDX         -1.058886
MCATOT          -1.442498
dtype: float64


#### Using pymatch library  (Python package designed for propensity score matching)

In [65]:
!pip install pymatch --quiet

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Define continuous and categorical variables
continuous_vars = ['MCATOT', 'NP3TOT', 'PDTIME(Years)', 'AgePDDX']
categorical_vars = ['SEX']

# Ensure continuous variables are numeric
for var in continuous_vars:
    OurPDDataforPM[var] = pd.to_numeric(OurPDDataforPM[var], errors='coerce')
    MOCA_DemoPDDX_UPDRS[var] = pd.to_numeric(MOCA_DemoPDDX_UPDRS[var], errors='coerce')

# Encode categorical variables if needed
le = LabelEncoder()
OurPDDataforPM['SEX'] = le.fit_transform(OurPDDataforPM['SEX'])
MOCA_DemoPDDX_UPDRS['SEX'] = le.transform(MOCA_DemoPDDX_UPDRS['SEX'])

# Combine datasets for standardization
combined_data = pd.concat([OurPDDataforPM[continuous_vars], MOCA_DemoPDDX_UPDRS[continuous_vars]], axis=0)

# Initialize and apply the StandardScaler
scaler = StandardScaler()
combined_data[continuous_vars] = scaler.fit_transform(combined_data[continuous_vars])

# Split combined data back into original datasets
OurPDDataforPM[continuous_vars] = combined_data.iloc[:len(OurPDDataforPM)]
MOCA_DemoPDDX_UPDRS[continuous_vars] = combined_data.iloc[len(OurPDDataforPM):]

In [ ]:
OurPDDataforPM.head()

In [ ]:
MOCA_DemoPDDX_UPDRS.head()

In [ ]:
# Combine the datasets and create a label column
OurPDDataforPM['label'] = 1  # Treatment  
MOCA_DemoPDDX_UPDRS['label'] = 0  # Control